In [1]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

## Current simulation 

In [4]:
current_run_units_added_sql = '''
SELECT  [units_index]
      ,[parcel_id]
      ,[units_added]
      ,[year_simulation]
      ,[run_id]
  FROM [urbansim].[urbansim].[urbansim_lite_output_units]
  where run_id = 1'''
units_added_df =  pd.read_sql(current_run_units_added_sql, mssql_engine)
units_added_df.drop('units_index',inplace=True,axis=1)
units_added_df.drop('run_id',inplace=True,axis=1)
units_added_df.sort_values(by=['parcel_id','year_simulation'],inplace=True)
units_added_df.set_index('parcel_id',inplace=True)

## Previous simulation 

In [5]:
previous_run_units_sql = '''SELECT  [units_index]
      ,[parcel_id]
      ,[units_added]
      ,[year_simulation]
      ,[run_id]
  FROM [urbansim].[urbansim].[urbansim_lite_output_units]
  where run_id = 2'''
previous_run =  pd.read_sql(previous_run_units_sql, mssql_engine)
previous_run.drop('run_id',inplace=True,axis=1)
previous_run.drop('units_index',inplace=True,axis=1)
previous_run.rename(columns = {'year_built': 'year_simulation'},inplace=True)
previous_run.rename(columns = {'residential_units': 'units_added'},inplace=True)
previous_run.sort_values(by=['parcel_id','year_simulation'],inplace=True)
previous_run.set_index('parcel_id',inplace=True)

## Comparison of simulations

In [6]:
print "\nDo the simulations produce the same results?"
print previous_run.equals(units_added_df)


Do the simulations produce the same results?
False


## Units added by jurisdiction

In [7]:
parcels_sql = '''
  WITH bldgs_by_parcel AS (SELECT parcel_id, SUM(residential_units) AS residential_units, 
                                  count(building_id) AS num_of_bldgs
                           FROM   urbansim.urbansim.building GROUP BY parcel_id)
  SELECT parcels.parcel_id, parcels.jurisdiction_id, parcels.site_id,
         parcels.capacity, 
         COALESCE(bldgs_by_parcel.residential_units,0) AS residential_units,
         COALESCE(bldgs_by_parcel.num_of_bldgs,0) AS bldgs,
         0 as partial_build
  FROM urbansim.urbansim.parcel parcels
  LEFT JOIN bldgs_by_parcel 
  ON bldgs_by_parcel.parcel_id = parcels.parcel_id
  WHERE parcels.capacity > 0
'''
parcels_df =  pd.read_sql(parcels_sql, mssql_engine,index_col='parcel_id')

In [8]:
sched_dev_sql = '''
select jurisdiction_id, sum(capacity) as schdev
from urbansim.urbansim.parcel
where  site_id is NOT NULL
group by jurisdiction_id
order by jurisdiction_id
'''
sched_dev =  pd.read_sql(sched_dev_sql, mssql_engine,index_col='jurisdiction_id')
# print sched_dev

In [9]:
capacity_sql = '''
SELECT jurisdiction_id,SUM([capacity]) AS capacity
FROM [urbansim].[urbansim].[parcel]
GROUP BY jurisdiction_id
ORDER BY jurisdiction_id
'''
capacity_df =  pd.read_sql(capacity_sql, mssql_engine,index_col='jurisdiction_id')

In [10]:
jurisdictions_sql = '''
  SELECT jurisdiction_id, name
  FROM urbansim.ref.jurisdiction
'''
jur_df =  pd.read_sql(jurisdictions_sql, mssql_engine,index_col='jurisdiction_id')

In [11]:
units_added_df.reset_index(inplace=True)

In [12]:
# get jurisdiction id for parcels

units_added_w_city = units_added_df.join(parcels_df,on='parcel_id')
units_by_city = pd.DataFrame({'sr14_sans_schdev': units_added_w_city .
                                            groupby(["jurisdiction_id"]).units_added.sum()})
forecast_sr14_df = units_by_city.join(sched_dev)
forecast_sr14_df = forecast_sr14_df.join(capacity_df)
forecast_sr14_df = forecast_sr14_df.join(jur_df)
forecast_sr14_df['forecast_sr14'] = forecast_sr14_df['sr14_sans_schdev'] + forecast_sr14_df['schdev']
forecast_sr14_df['remaining'] = forecast_sr14_df['capacity'] - forecast_sr14_df['forecast_sr14']
forecast_sr14_df_totals = forecast_sr14_df.copy()
forecast_sr14_df_totals.reset_index(inplace=True)
forecast_sr14_df_totals['jurisdiction_id'] = forecast_sr14_df_totals['jurisdiction_id'].astype(str)
sr14_totals = forecast_sr14_df_totals.append(forecast_sr14_df_totals.sum(numeric_only=True), ignore_index=True)
sr14_totals.at[19, 'jurisdiction_id'] = 'totals'
sr14_totals.set_index('jurisdiction_id',inplace=True)
cols = ['sr14_sans_schdev','schdev','capacity','forecast_sr14','remaining']
sr14_totals[cols] = sr14_totals[cols].apply(pd.to_numeric, errors='coerce', axis=1,downcast='integer')
sr14_totals[['name','capacity','sr14_sans_schdev','schdev','forecast_sr14','remaining']]


,name,capacity,sr14_sans_schdev,schdev,forecast_sr14,remaining
jurisdiction_id,,,,,,
1,Carlsbad,6692,3910,2782,6692,0
2,Chula Vista,27215,8460,18755,27215,0
3,Coronado,246,187,36,223,23
4,Del Mar,53,32,2,34,19
5,El Cajon,3178,2828,350,3178,0
6,Encinitas,2460,2197,249,2446,14
7,Escondido,10933,8414,2519,10933,0
8,Imperial Beach,3569,2628,228,2856,713
9,La Mesa,9307,8692,615,9307,0


## Compare to series 13

In [13]:
sr13_forecast_sql = ''' 
WITH sr13_capacity AS (
SELECT  City,sum([hs]) as base_yr_hs_sr13
      ,sum([cap_hs]) as capacity_sr13
  FROM [regional_forecast].[sr13_final].[capacity] x
  inner join [regional_forecast].[sr13_final].[mgra13] as y 
  on y.mgra = x.mgra
   where scenario = 0 and increment = 2012 --and y.City = 1
   GROUP BY City
),
sr13_2050 AS (
SELECT  y.jurisdiction_id,yr_id, sum(units) as units2050
FROM [demographic_warehouse].[fact].[housing] as x
inner join [demographic_warehouse].[dim].[mgra_denormalize] as y 
on x.mgra_id = y.mgra_id
where units > 0 and x.datasource_id = 13 and x.yr_id = 2050
group by yr_id,y.jurisdiction_id
),
sr13_2012 AS (
SELECT  y.jurisdiction_id,yr_id, sum(units) as units2012
FROM [demographic_warehouse].[fact].[housing] as x
inner join [demographic_warehouse].[dim].[mgra_denormalize] as y 
on x.mgra_id = y.mgra_id
where units > 0 and x.datasource_id = 13 and x.yr_id = 2012
group by yr_id,y.jurisdiction_id
)
SELECT sr13_2012.jurisdiction_id,base_yr_hs_sr13,capacity_sr13,sr13_2050.units2050 - sr13_2012.units2012 as forecast_sr13
FROM sr13_2012 
JOIN sr13_2050
ON sr13_2012.jurisdiction_id = sr13_2050.jurisdiction_id
JOIN sr13_capacity
ON sr13_capacity.City = sr13_2012.jurisdiction_id
order by sr13_2012 .jurisdiction_id
'''
sr13_forecast =  pd.read_sql(sr13_forecast_sql, mssql_engine,index_col='jurisdiction_id')

In [15]:
forecast_compare = forecast_sr14_df.join(sr13_forecast)
forecast_compare['diff'] =  forecast_compare['forecast_sr14'] - forecast_compare['forecast_sr13']
forecast_compare.reset_index(inplace=True)
forecast_compare['jurisdiction_id'] = forecast_compare['jurisdiction_id'].astype(str)
x = forecast_compare.append(forecast_compare.sum(numeric_only=True), ignore_index=True)
x.at[19, 'jurisdiction_id'] = 'totals'
x.set_index('jurisdiction_id',inplace=True)
cols = ['forecast_sr14','forecast_sr13','diff']
x[cols] = x[cols].apply(pd.to_numeric, errors='coerce', axis=1,downcast='integer')
x[['name','forecast_sr14','forecast_sr13','diff']]

,name,forecast_sr14,forecast_sr13,diff
jurisdiction_id,,,,
1,Carlsbad,6692,5334,1358
2,Chula Vista,27215,29018,-1803
3,Coronado,223,205,18
4,Del Mar,34,37,-3
5,El Cajon,3178,4875,-1697
6,Encinitas,2446,2100,346
7,Escondido,10933,7701,3232
8,Imperial Beach,2856,1665,1191
9,La Mesa,9307,7567,1740


## Number of units added in simulation:

In [ ]:
print previous_run.units_added.sum()
print units_added_df.units_added.sum()

In [ ]:
# duplicated parcel ids (developed over more than one year)
duplicated_parcels =  units_added_df[units_added_df.duplicated(['parcel_id'],keep=False)].sort_values(by='parcel_id')
print duplicated_parcels.head()
duplicated_parcels_count = pd.DataFrame({'count_parcels': duplicated_parcels.groupby(["parcel_id"]).size()})
duplicated_parcels_count.sort_values(by='count_parcels',inplace=True)

In [ ]:
duplicated_parcels_count['count_sum'] = duplicated_parcels_count.count_parcels -  1
print('\nNumber of parcels with units built over multiple years:')
print len(duplicated_parcels_count)
print('\nNumber of extra rows (some parcels have units added in 2 years, some in 3 yrs:')
print duplicated_parcels_count['count_sum'].sum()
# print duplicated_parcels_count.head()

In [ ]:
units_by_parcel = pd.DataFrame({'total_units_added': units_added_df.groupby(["parcel_id"]).
                                units_added.sum()}).reset_index()

In [ ]:
units_by_parcel.set_index('parcel_id',inplace=True)
print units_by_parcel.head()

In [ ]:
print len(units_by_parcel)
print len(units_added_df)
# print len(units_added_df) - duplicated_parcels_count.count_parcels.sum() 
print len(units_added_df) - duplicated_parcels_count['count_sum'].sum()

In [ ]:
print units_by_parcel.head()

In [ ]:
print "Number of parcels with capacity that have no units added:"
print len(parcels_df[~parcels_df.index.isin(units_by_parcel.index)].reset_index())

In [ ]:
print "Number of parcels that have units added but no capacity:"
print len(units_by_parcel[~units_by_parcel.index.isin(parcels_df.index)].reset_index())

In [ ]:
print parcels_df.head()
#print units_by_parcel.head()

In [ ]:
parcels_plus_capacity = parcels_df.join(units_by_parcel)
print parcels_plus_capacity.head()

In [ ]:
parcels_plus_capacity['diff'] = parcels_plus_capacity['capacity'] - parcels_plus_capacity['total_units_added']

In [ ]:
parcels_plus_capacity.to_csv('data/pcap.csv')

In [ ]:
households_sql = '''
  SELECT sum(hh) AS hh,yr
  FROM isam.demographic_output.summary
  WHERE sim_id = 1004 and yr > 2019
  GROUP BY yr
'''

In [ ]:
buildings_sql = '''
SELECT  SUM(COALESCE(residential_units,0)) AS residential_units
FROM urbansim.urbansim.building
'''

In [ ]:
sched_dev_sql = '''
SELECT  SUM(COALESCE(capacity,0)) 
FROM urbansim.urbansim.parcel
WHERE site_id is NOT NULL and capacity > 0
'''

In [ ]:
hh_df =  pd.read_sql(households_sql, mssql_engine)
du_df =  pd.read_sql(buildings_sql, mssql_engine)
sh_df =  pd.read_sql(sched_dev_sql, mssql_engine)

In [ ]:
hh = hh_df.loc[hh_df.yr==2050].hh.values[0]
du = int(du_df.values)
sched_dev_capacity = int(sh_df.values)

In [ ]:
units_needed = hh - du - sched_dev_capacity

In [ ]:
print '\nHouseholds 2050:'
print hh
print '\nResidential units base year:'
print du
print '\nSched dev:'
print sched_dev_capacity
print '\nUnits needed = Households 2050 - Residential units base year - Sched dev'
print '\nUnits needed:'
print units_needed
print '\nTotal units added:'
print int(units_added_df.units_added.sum())

In [ ]:
hh_diff = hh_df.set_index('yr').diff()
hh_diff.reset_index(inplace=True)
ts = pd.Series(hh_diff['hh'].values, index=hh_diff['yr'])

In [ ]:
# Number of new households by year 
ts.plot.bar()

In [ ]:
# Number of units added by year
df = units_added_df.groupby(['year_simulation'])['units_added'].sum()
df.plot.bar()

In [ ]:
units_added_df.sort_values(by='units_added',inplace=True,ascending=False)

In [ ]:
print units_added_df.head(10)

In [ ]:
print units_added_df.loc[units_added_df.parcel_id==9002470]

In [ ]:
# Number of Total households by year
df = hh_df.groupby(['yr'])['hh'].sum()
df.plot.bar()